In [1]:
pip install paddlepaddle

     |████████████████████████████████| 204.9MB 71kB/s 


In [2]:
pip install paddle-ernie

  Created wheel for paddle-ernie: filename=paddle_ernie-0.1.0.dev1-cp37-none-any.whl size=20016 sha256=b6f4d6f96a358056bf36bb94941d28e18ecab8f4c028bd9ecec659cdbd3942fd
  Stored in directory: /root/.cache/pip/wheels/ef/d8/52/d0fc0e19bce14963259ada61138e6c520805bd688e29652796
Successfully built paddle-ernie


In [3]:
import sys
!git clone https://github.com/PaddlePaddle/ERNIE.git
!git -C ./ERNIE branch origin/dygraph
sys.path.append('./ERNIE')
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import paddle as P
import paddle.fluid as F
import paddle.fluid.layers as L
import paddle.fluid.dygraph as D

from ernie.tokenizing_ernie import ErnieTokenizer
from ernie.modeling_ernie import ErnieModelForSequenceClassification

Cloning into 'ERNIE'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 2034 (delta 1), reused 3 (delta 1), pack-reused 2027
Receiving objects: 100% (2034/2034), 88.86 MiB | 37.51 MiB/s, done.
Resolving deltas: 100% (1106/1106), done.


In [4]:
df_train_text = pd.read_csv("https://raw.githubusercontent.com/zeitgeist-hash/tweeteval/main/datasets/emotion/train_text.txt", delimiter="\t", names=["Tweet"])

In [5]:
df_train_label = pd.read_csv("https://raw.githubusercontent.com/zeitgeist-hash/tweeteval/main/datasets/emotion/train_labels.txt", delimiter="\t", names=["Label"])

In [6]:
df_train = df_train_text.join(df_train_label)
df_train.to_csv("train.csv", encoding='utf-8', index=False)
df_train

,Tweet,Label
0,“Worry is a down payment on a problem you may ...,2
1,My roommate: it's okay that we can't spell bec...,0
2,No but that's so cute. Atsu was probably shy a...,1
3,Rooneys fucking untouchable isn't he? Been fuc...,0
4,it's pretty depressing when u hit pan on ur fa...,3
...,...,...
3252,I get discouraged because I try for 5 fucking ...,3
3253,The @user are in contention and hosting @user ...,3
3254,@user @user @user @user @user as a fellow UP g...,0
3255,You have a #problem? Yes! Can you do #somethin...,0


In [7]:
df_test_text = pd.read_csv("https://raw.githubusercontent.com/zeitgeist-hash/tweeteval/main/datasets/emotion/test_text.txt", delimiter="\t", names=["Tweet"])

In [8]:
df_test_label = pd.read_csv("https://raw.githubusercontent.com/zeitgeist-hash/tweeteval/main/datasets/emotion/test_labels.txt", delimiter="\t", names=["Label"])

In [9]:
df_test = df_test_text.join(df_test_label)
df_test.to_csv("test.csv", encoding='utf-8', index=False)
df_test

,Tweet,Label
0,#Deppression is real. Partners w/ #depressed p...,3
1,@user Interesting choice of words... Are you c...,0
2,My visit to hospital for care triggered #traum...,3
3,@user Welcome to #MPSVT! We are delighted to h...,1
4,What makes you feel #joyful?,1
...,...,...
1416,I need a sparkling bodysuit . No occasion. Jus...,1
1417,@user I've finished reading it; simply mind-bl...,3
1418,shaft abrasions from panties merely shifted to...,0
1419,All this fake outrage. Y'all need to stop 🤣,0


In [10]:
#设置好所有的超参数，对于ERNIE任务学习率推荐取1e-5/2e-5/5e-5, 根据显存大小调节BATCH大小, 最大句子长度不超过512.
BATCH=32
MAX_SEQLEN=300
LR=5e-5
EPOCH=10

In [11]:
D.guard().__enter__() # 为了让Paddle进入动态图模式，需要添加这一行在最前面

ernie = ErnieModelForSequenceClassification.from_pretrained('ernie-2.0-en', num_labels=4)
optimizer = F.optimizer.Adam(LR, parameter_list=ernie.parameters())
tokenizer = ErnieTokenizer.from_pretrained('ernie-2.0-en')

downloading https://ernie-github.cdn.bcebos.com/model-ernie2.0-en.1.tar.gz: 873767KB [01:14, 11664.35KB/s]
/usr/local/lib/python3.7/dist-packages/ernie/modeling_ernie.py:265: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  log.warn('param:%s not set in pretrained model, skip' % k)
param:classifier.weight not set in pretrained model, skip
param:classifier.bias not set in pretrained model, skip


In [12]:
def make_data(path):
    data = []
    for i, l in enumerate(open(path)):
        if i == 0:
            continue
        l = l.strip().split(',')
        text, label = l[0], int(l[-1])
        text_id, _ = tokenizer.encode(text)
        text_id = text_id[:MAX_SEQLEN]
        text_id = np.pad(text_id, [0, MAX_SEQLEN-len(text_id)], mode='constant')
        label_id = np.array(label+1)
        data.append((text_id, label_id))
    return data

In [13]:
train_data = make_data('/content/train.csv')

In [14]:
train_data

Output hidden; open in https://colab.research.google.com to view.

In [15]:
test_data = make_data('/content/test.csv')

In [16]:
test_data

Output hidden; open in https://colab.research.google.com to view.

In [17]:
def get_batch_data(data, i):
    d = data[i*BATCH: (i + 1) * BATCH]
    feature, label = zip(*d)
    feature = np.stack(feature)  # 将BATCH行样本整合在一个numpy.array中
    label = np.stack(list(label))
    feature = D.to_variable(feature) # 使用to_variable将numpy.array转换为paddle tensor
    label = D.to_variable(label)
    return feature, label

In [ ]:
for i in range(EPOCH):
    np.random.shuffle(train_data) # 每个epoch都shuffle数据以获得最佳训练效果；
    #train
    for j in range(len(train_data) // BATCH):
        feature, label = get_batch_data(train_data, j)
        loss, _ = ernie(feature, labels=label) # ernie模型的返回值包含(loss, logits)；其中logits目前暂时不需要使用
        loss.backward()
        optimizer.minimize(loss)
        ernie.clear_gradients()
        if j % 10 == 0:
            print('train %d: loss %.5f' % (j, loss.numpy()))
        # evaluate
        if j % 100 == 0:
            all_pred, all_label = [], []
            with D.base._switch_tracer_mode_guard_(is_train=False): # 在这个with域内ernie不会进行梯度计算；
                ernie.eval() # 控制模型进入eval模式，这将会关闭所有的dropout；
                for j in range(len(test_data) // BATCH):
                    feature, label = get_batch_data(test_data, j)
                    loss, logits = ernie(feature, labels=label) 
                    all_pred.extend(L.argmax(logits, -1).numpy())
                    all_label.extend(label.numpy())
                ernie.train()
            f1 = f1_score(all_label, all_pred, average='macro')
            print('f1 %.5f' % f1)